
#### Basic Neural Network, first:

0: Importing the libraries  <br>
1: Fully connected Network <br>
2: device <br>
3: hyperparameters <br>

#### Then
4: Load Data <br>
5: Initialize model/network <br>
6: configure loss and optimizer <br>
7: Training loop <br>
8: Test <br>


In [ ]:
# 0 Importing the libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
# 1 Creating a Fully Connected Network
class BasicNeuralNetwork(nn.Module):
    def __init__(self, input_size, num_classes): #input size will be 28*28=784, num_classes will be 10 for mnist
        super(BasicNeuralNetwork, self).__init__() # calls the init method of the parent class (nn.Module), we run the init of that method.
        #for a small network, we can do two laters:
        self.l1 = nn.Linear(input_size,50) #50 neurons in the first layer
        self.l2 = nn.Linear(50 , num_classes) #num_classes neurons in the second layer

    #define the forward method that will be used to pass the data through the network
    def forward(self, x):
        x = F.relu(self.l1(x)) #relu activation function
        x = self.l2(x)
        return x

In [ ]:
# testing:

model = BasicNeuralNetwork(784, 10)
x = torch.randn(64, 784)
print(model(x).shape)

torch.Size([64, 10])


In [ ]:
# 2 Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# hyperparameters:
input_size = 784
num_classes = 10
learning_rate = 0.001 #this means that we will update the weights by 0.001*gradient at each step of the optimization process (backpropagation)
batch_size = 64
num_epochs = 1 #one epoch is when the entire dataset is passed through the network once

In [ ]:
# 3 Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Failed to download (trying next):
HTTP Error 404: Not Found

Extracting dataset/MNIST\raw\train-images-idx3-ubyte.gz to dataset/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found

Extracting dataset/MNIST\raw\train-labels-idx1-ubyte.gz to dataset/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found

Extracting dataset/MNIST\raw\t10k-images-idx3-ubyte.gz to dataset/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found

Extracting dataset/MNIST\raw\t10k-labels-idx1-ubyte.gz to dataset/MNIST\raw



In [ ]:
# 4 Initialize network
model = BasicNeuralNetwork(
    input_size=input_size, 
    num_classes=num_classes).to(device)

In [ ]:
# 5 Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# 6 Train Network
for epoch in range(num_epochs):
    for batch_index, (data,targets) in enumerate(train_loader): #enumerate gives us the index of the batch, and the data and targets. data is the images, targets are the labels
        # make the data to the device
        data = data.to(device=device)
        targets = targets.to(device=device)
        # Flatten the data
        data = data.reshape(data.shape[0],-1)
        
        # now for forward pass:
        scores = model(data) #pass the data through the model
        loss = criterion(scores, targets) #calculate the loss
        
        #backward pass:
        optimizer.zero_grad() #set the gradients to zero before calculating the gradients
        loss.backward() #calculate the gradients
        optimizer.step() #update the weights (gradient descent step)
        

In [ ]:
# 7 Check accuracy on training & test to see how good our model is

def check_accuracy(loader, model):
    if(loader.dataset.train):
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')
    num_correct = 0
    num_samples = 0
    model.eval() #tells pytorch that we are in evaluation mode
    
    with torch.no_grad(): #we don't need to calculate the gradients when we are evaluating the model
        for x,y in loader:
            x= x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x) # pass the data through the model
            _, predictions = scores.max(1) #get the index of the max value, which in this case is the predicted class label (0-9), that has the highest score
            num_correct += (predictions == y).sum() #sum the number of correct predictions
            num_samples += predictions.size(0) #total number of samples, which is the batch size
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    
    
    model.train() #put the model back to training mode
    
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)
        

Checking accuracy on training data
Got 56148 / 60000 with accuracy 93.58
Checking accuracy on test data
Got 9359 / 10000 with accuracy 93.59
